# Only for real nerds

## Create a preprocessing workflow that includes Realignment, Coregistration, Normalization and Smoothing

### Setup 

In [25]:
# import all the stuff that you need
from nipype.interfaces import spm
from nipype import Workflow, Node
import os

# get current working directory
working_directory = os.getcwd()

# configure spm
spm.SPMCommand.set_mlab_paths(matlab_cmd='/opt/matlab/R2022a/bin/matlab', paths='/opt/matlab/toolboxes/spm12', use_mcr=False)

# get paths to functional and anatomical images
fmri_img = os.path.join(working_directory,'data/fmri_img.nii')
anatomical_img = os.path.join(working_directory,'data/spm_auditory/sub001/sM00223/sM00223_002.img')

stty: 'standard input': Inappropriate ioctl for device


### Set up all the nodes we need for preprocessing

In [68]:
# Use Realign for motion correction and register all images to the mean image. Since this is the first node, we have to provide with the filepath 
realign = Node(interface=spm.Realign(in_files=[fmri_img],jobtype='estwrite'), name="realign")

# Use Coregister to perform a rigid body registration of the functional data to the structural data. We have to provide it with the anatomical image
coregister = Node(interface=spm.Coregister(source=[anatomical_img]), name="coregister")

# Segment the coregistered structural image
segment = Node(interface=spm.NewSegment(channel_info=(0.0001,60,(True,True)),write_deformation_fields=[False,True]), name="segment")
tpm_img ='/opt/matlab/toolboxes/spm12/tpm/TPM.nii'
tissue1 = ((tpm_img, 1), 1, (True,False), (False, False))
tissue2 = ((tpm_img, 2), 1, (True,False), (False, False))
tissue3 = ((tpm_img, 3), 2, (True,False), (False, False))
tissue4 = ((tpm_img, 4), 3, (False,False), (False, False))
tissue5 = ((tpm_img, 5), 4, (False,False), (False, False))
tissue6 = ((tpm_img, 6), 2, (False,False), (False, False))
tissues = [tissue1, tissue2, tissue3, tissue4, tissue5, tissue6]
segment.inputs.tissues = tissues

# Warp functional and structural data to SPM’s T1 template using Normalize.
normalize = Node(interface=spm.Normalize12(), name="normalize")

# Smooth the functional data using Smooth
smooth = Node(interface=spm.Smooth(), name="smooth")

stty: 'standard input': Inappropriate ioctl for device


### Setup the workflow and run it

In [71]:
preprocessing_wf = Workflow(name='preprocessing_wf',base_dir='.')
preprocessing_wf.connect(realign,'mean_image',coregister,'target')
preprocessing_wf.connect(coregister,'coregistered_source',segment,'channel_files')
preprocessing_wf.run()

230421-17:52:24,828 nipype.workflow INFO:
	 Workflow preprocessing_wf settings: ['check', 'execution', 'logging', 'monitoring']
230421-17:52:24,831 nipype.workflow INFO:
	 Running serially.
230421-17:52:24,831 nipype.workflow INFO:
	 [Node] Setting-up "preprocessing_wf.realign" in "/home/johannes.wiesner/work/repos/workshop_cimh/workshop/courses/nipype/preprocessing_wf/realign".
230421-17:52:24,833 nipype.workflow INFO:
	 [Node] Cached "preprocessing_wf.realign" - collecting precomputed outputs
230421-17:52:24,833 nipype.workflow INFO:
	 [Node] "preprocessing_wf.realign" found cached.
230421-17:52:24,833 nipype.workflow INFO:
	 [Node] Setting-up "preprocessing_wf.coregister" in "/home/johannes.wiesner/work/repos/workshop_cimh/workshop/courses/nipype/preprocessing_wf/coregister".
230421-17:52:24,835 nipype.workflow INFO:
	 [Node] Cached "preprocessing_wf.coregister" - collecting precomputed outputs
230421-17:52:24,835 nipype.workflow INFO:
	 [Node] "preprocessing_wf.coregister" found ca

stty: 'standard input': Inappropriate ioctl for device


In [53]:
seg = spm.NewSegment()
seg.inputs

stty: 'standard input': Inappropriate ioctl for device



affine_regularization = <undefined>
channel_files = <undefined>
channel_info = <undefined>
matlab_cmd = /opt/matlab/R2022a/bin/matlab
mfile = True
paths = ['/opt/matlab/toolboxes/spm12']
sampling_distance = <undefined>
tissues = <undefined>
use_mcr = <undefined>
use_v8struct = True
warping_regularization = <undefined>
write_deformation_fields = <undefined>